In [1]:
# First, let's import necessary modules from TensorFlow
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # To make TensorFlow v1 code compatible with v2

# Assume the model functions are already defined
from model import DecomNet,Illumination_adjust_curve_net_ratio,Restoration_net

# Now let's try to build the model with a random input
# Replace the real input image with a random tensor
training = tf.placeholder_with_default(False, shape=(), name='training')
input_image = tf.random.normal([1, 256, 256, 3])  # Assume the input image size is 256x256x3
input_decom = tf.placeholder(tf.float32, [None, None, None, 3], name='input_decom')
input_low_r = tf.placeholder(tf.float32, [None, None, None, 3], name='input_low_r')
input_low_i = tf.placeholder(tf.float32, [None, None, None, 1], name='input_low_i')
input_low_i_ratio = tf.placeholder(tf.float32, [None, None, None, 1], name='input_low_i_ratio')
# # Call the model building functions
# R_low, I_low = DecomNet(input_image)
# output_i, A = Illumination_adjust_curve_net_ratio(input_low_i, input_low_i_ratio)
# output_r = Restoration_net(input_low_r, input_low_i, training)
# # Get the output nodes' names
# R_low_name = R_low.op.name
# I_low_name = I_low.op.name

def KinD_LCE(input_decom, input_low_r, input_low_i, input_low_i_ratio, training):
    [R_decom, I_decom] = DecomNet(input_decom)
    decom_output_R = R_decom
    decom_output_I = I_decom
    output_i, A = Illumination_adjust_curve_net_ratio(input_low_i, input_low_i_ratio)
    output_r = Restoration_net(input_low_r, input_low_i, training)
    
    return output_i, output_r, decom_output_R, decom_output_I
output_i, output_r, decom_output_R, decom_output_I = KinD_LCE(input_decom, input_low_r, input_low_i, input_low_i_ratio, training)

# print(f"layer names:{R_low_name}, {I_low_name},{output_i.op.name},{output_r.op.name}")
print(f"kind_layer:{output_i.op.name},{output_r.op.name},{decom_output_R.op.name},{decom_output_I.op.name}")

Instructions for updating:
non-resource variables are not supported in the long term


/home/nvidia/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Instructions for updating:
Colocations handled automatically by placer.


/home/nvidia/Desktop/PommesPeter/lowlight-enhancement-research/KinD_plus/msia_BN_3_M.py:13: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  concat = tf.layers.conv2d(input_i,
/home/nvidia/Desktop/PommesPeter/lowlight-enhancement-research/KinD_plus/msia_BN_3_M.py:33: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  pu_conv = tf.layers.batch_normalization(pu_conv, training=training)
/home/nvidia/Desktop/PommesPeter/lowlight-enhancement-research/KinD_plus/msia_BN_3_M.py:40: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.c

kind_layer:I_enhance_Net_ratio/Sigmoid,Denoise_Net/Sigmoid,DecomNet/Sigmoid,DecomNet/Sigmoid_1


In [10]:
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
import os
import numpy as np
from skimage import filters,color

# Assume the model functions are already defined
from model import DecomNet, Illumination_adjust_curve_net_ratio, Restoration_net

def KinD_LCE(input_decom, input_low_r, input_low_i, input_low_i_ratio, training):
    [R_decom, I_decom] = DecomNet(input_decom)
    decom_output_R = R_decom
    decom_output_I = I_decom
    output_i, A = Illumination_adjust_curve_net_ratio(input_low_i, input_low_i_ratio)
    output_r = Restoration_net(input_low_r, input_low_i, training)
    
    return output_i, output_r, decom_output_R, decom_output_I

# Create a session and make it the default session
sess = tf.Session()
tf.keras.backend.set_session(sess)

# Build the model
input_decom = tf.placeholder(tf.float32, [None, None, None, 3], name='input_decom')
input_low_r = tf.placeholder(tf.float32, [None, None, None, 3], name='input_low_r')
input_low_i = tf.placeholder(tf.float32, [None, None, None, 1], name='input_low_i')
input_low_i_ratio = tf.placeholder(tf.float32, [None, None, None, 1], name='input_low_i_ratio')
training = tf.placeholder_with_default(False, shape=(), name='training')

# Call the model building function


# Assume you have trained the model and the weights are loaded

# We will save the model in the Protobuf format
print(f"outputname:{output_i.op.name}")
print(f"outputname:{output_r.op.name}")
print(f"outputname:{decom_output_R.op.name}")
print(f"outputname:{decom_output_I.op.name}")
output_node_names = ['I_enhance_Net_ratio/Sigmoid', 'Denoise_Net/Sigmoid', 'DecomNet/Sigmoid', 'DecomNet/Sigmoid_1']
input_node_names = ['input_decom', 'input_low_r', 'input_low_i', 'input_low_i_ratio', 'training']
output_graph_name = './model.pb'
checkpoint_dir = './experiment/exp2/checkpoint'
illmin_name = "illumination_adjust_curve_net_global_rm_del_rotate"

var_Decom = [var for var in tf.trainable_variables() if 'DecomNet' in var.name]
var_adjust = [var for var in tf.trainable_variables() if 'I_enhance_Net' in var.name]
var_restoration = [var for var in tf.trainable_variables() if 'Denoise_Net' in var.name]
g_list = tf.global_variables()
bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
var_restoration += bn_moving_vars
saver_Decom = tf.train.Saver(var_list=var_Decom)
saver_adjust = tf.train.Saver(var_list=var_adjust)
saver_restoration = tf.train.Saver(var_list=var_restoration)
decom_checkpoint_dir = os.path.join(checkpoint_dir, 'decom_net_retrain')
ckpt_pre = tf.train.get_checkpoint_state(decom_checkpoint_dir)
if ckpt_pre:
    print('[*] loaded ' + ckpt_pre.model_checkpoint_path)
    saver_Decom.restore(sess, ckpt_pre.model_checkpoint_path)
else:
    print('[*] No decomnet pretrained model!')

checkpoint_dir_adjust = os.path.join(checkpoint_dir, illmin_name)
ckpt_adjust = tf.train.get_checkpoint_state(checkpoint_dir_adjust)
if ckpt_adjust:
    print('[*] loaded ' + ckpt_adjust.model_checkpoint_path)
    saver_adjust.restore(sess, ckpt_adjust.model_checkpoint_path)
else:
    print("[*] No adjust net pretrained model!")

checkpoint_dir_restoration = os.path.join(checkpoint_dir, 'new_restoration_retrain')
ckpt = tf.train.get_checkpoint_state(checkpoint_dir_restoration)
if ckpt:
    print('[*] loaded ' + ckpt.model_checkpoint_path)
    saver_restoration.restore(sess, ckpt.model_checkpoint_path)
else:
    print("[*] No restoration net pretrained model!")

@tf.function
def infer(input_low,training, input_decom, input_low_r, input_low_i, input_low_i_ratio):
    input_low_eval = tf.expand_dims(input_low, axis=0)
    h, w, _ = input_low.shape.as_list()
    ratio = float(ratio)
    
    output_i, output_r, decom_output_R, decom_output_I = KinD_LCE(input_decom, input_low_r, input_low_i, input_low_i_ratio, training)
    decom_r_low, decom_i_low = sess.run([decom_output_R, decom_output_I], feed_dict={input_decom: input_low_eval})
    restoration_r = sess.run(output_r, feed_dict={input_low_r: decom_r_low, input_low_i: decom_i_low, training: False})
    # change the ratio to get different exposure level, the value can be 0-5.0
        
    i_low_data_ratio = np.ones([h, w]) * ratio
    i_low_ratio_expand = np.expand_dims(i_low_data_ratio, axis=2)
    i_low_ratio_expand2 = np.expand_dims(i_low_ratio_expand, axis=0)
    adjust_i = sess.run(output_i, feed_dict={input_low_i: decom_i_low, input_low_i_ratio: i_low_ratio_expand2})
    
    # The restoration result can find more details from very dark regions, however, it will restore the very dark regions
    # with gray colors, we use the following operator to alleviate this weakness.
    decom_r_sq = np.squeeze(decom_r_low)
    r_gray = color.rgb2gray(decom_r_sq)
    r_gray_gaussion = filters.gaussian(r_gray, 3)
    low_i = np.minimum((r_gray_gaussion * 2) ** 0.5, 1)
    low_i_expand_0 = np.expand_dims(low_i, axis=0)
    low_i_expand_3 = np.expand_dims(low_i_expand_0, axis=3)
    result_denoise = restoration_r * low_i_expand_3
    fusion4 = result_denoise * adjust_i
    
    return fusion4

signatures = {
    'serving_default': infer.get_concrete_function(
        tf.TensorSpec(shape=(), dtype=tf.bool, name="training"),
        # tf.TensorSpec(shape=[None, None, 3], dtype=tf.float32, name="input_low"),
        tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.float32, name="input_decom"),
        tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.float32, name="input_low_r"),
        tf.TensorSpec(shape=[None, None, None, 1], dtype=tf.float32, name="input_low_i"),
        tf.TensorSpec(shape=[None, None, None, 1], dtype=tf.float32, name="input_low_i_ratio")
    )
}

# Convert the function to a TensorFlow function
infer = tf.function(infer)

# Save the model
tf.saved_model.save(infer, "saved_model_dir", signatures=signatures)

# saver = tf.train.Saver()
# save_path = saver.save(sess, "./model/model.ckpt")

# # Convert to SavedModel
# builder = tf.saved_model.builder.SavedModelBuilder("./model/saved_model")
# builder.add_meta_graph_and_variables(sess,
#                                       [tf.saved_model.tag_constants.SERVING],
#                                       signature_def_map= {
#                                           "serving_default": tf.saved_model.signature_def_utils.predict_signature_def(
#                                               inputs= {"input_decom": input_decom, "input_low_r": input_low_r, "input_low_i": input_low_i, "input_low_i_ratio": input_low_i_ratio},
#                                               outputs= {"output_i": output_i, "output_r": output_r})
#                                       })
# builder.save() 

# # Convert variables to constants and save the model
# output_graph_def = tf.graph_util.convert_variables_to_constants(
#     sess,  # The session is used to retrieve the weights
#     tf.get_default_graph().as_graph_def(),  # The graph_def is used to retrieve the nodes
#     output_node_names  # The output node names are used to define the usefull nodes
# )

# # Finally we serialize and dump the output graph to the filesystem
# with tf.gfile.GFile(output_graph_name, "wb") as f:
#     f.write(output_graph_def.SerializeToString())

# print("%d ops in the final graph." % len(output_graph_def.node))

2023-08-05 16:04:40.653496: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 16:04:40.653949: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 16:04:40.654238: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 16:04:40.654657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:00:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-05 16:04:40.654773: I tensorflow/core/co

outputname:I_enhance_Net_ratio_4/Sigmoid
outputname:Denoise_Net_4/Sigmoid
outputname:DecomNet_4/Sigmoid
outputname:DecomNet_4/Sigmoid_1
[*] loaded ./experiment/exp2/checkpoint/decom_net_retrain/Decomposition_Net.ckpt-150000
INFO:tensorflow:Restoring parameters from ./experiment/exp2/checkpoint/decom_net_retrain/Decomposition_Net.ckpt-150000


INFO:tensorflow:Restoring parameters from ./experiment/exp2/checkpoint/decom_net_retrain/Decomposition_Net.ckpt-150000


[*] loaded ./experiment/exp2/checkpoint/illumination_adjust_curve_net_global_rm_del_rotate/Illumination_Adjustment_Net.ckpt-112000
INFO:tensorflow:Restoring parameters from ./experiment/exp2/checkpoint/illumination_adjust_curve_net_global_rm_del_rotate/Illumination_Adjustment_Net.ckpt-112000


INFO:tensorflow:Restoring parameters from ./experiment/exp2/checkpoint/illumination_adjust_curve_net_global_rm_del_rotate/Illumination_Adjustment_Net.ckpt-112000


[*] loaded ./experiment/exp2/checkpoint/new_restoration_retrain/Reflectance_Restoration_Net.ckpt-2399
INFO:tensorflow:Restoring parameters from ./experiment/exp2/checkpoint/new_restoration_retrain/Reflectance_Restoration_Net.ckpt-2399


INFO:tensorflow:Restoring parameters from ./experiment/exp2/checkpoint/new_restoration_retrain/Reflectance_Restoration_Net.ckpt-2399


TypeError: in user code:


    TypeError: tf__infer() missing 1 required positional argument: 'input_low_i_ratio'


In [1]:
# For DecomNet
decom_signature = tf.saved_model.signature_def_utils.predict_signature_def(
    inputs={'input_decom': input_decom},
    outputs={'decom_output_R': decom_output_R, 'decom_output_I': decom_output_I})

# For I_enhance_Net
i_enhance_signature = tf.saved_model.signature_def_utils.predict_signature_def(
    inputs={'input_low_i': input_low_i, 'input_low_i_ratio': input_low_i_ratio},
    outputs={'output_i': output_i})

# For Denoise_Net
denoise_signature = tf.saved_model.signature_def_utils.predict_signature_def(
    inputs={'input_low_r': input_low_r, 'input_low_i': input_low_i},
    outputs={'output_r': output_r})

builder.add_meta_graph_and_variables(sess,
                                      [tf.saved_model.tag_constants.SERVING],
                                      signature_def_map={
                                          "decom": decom_signature,
                                          "i_enhance": i_enhance_signature,
                                          "denoise": denoise_signature})



NameError: name 'tf' is not defined

In [2]:
loaded_model = tf.saved_model.load(saved_model_dir)

# For DecomNet
decom_func = loaded_model.signatures['decom']
decom_output = decom_func(input_decom)

# For I_enhance_Net
i_enhance_func = loaded_model.signatures['i_enhance']
i_enhance_output = i_enhance_func(input_low_i, input_low_i_ratio)

# For Denoise_Net
denoise_func = loaded_model.signatures['denoise']
denoise_output = denoise_func(input_low_r, input_low_i)


NameError: name 'tf' is not defined

In [ ]:
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
import os
import numpy as np
from skimage import filters,color

# Assume the model functions are already defined
from model import DecomNet, Illumination_adjust_curve_net_ratio, Restoration_net


graph = tf.Graph()
with graph.as_default():
    # 占位符
    input_decom = tf.placeholder(tf.float32, [None, None, None, 3], name='input_decom')
    input_low_i_ratio = tf.placeholder(tf.float32, [None, None, None, 1], name='input_low_i_ratio')

    # 模型结构
    decom_r_low, decom_i_low = DecomNet(input_decom)
    restoration_r = Restoration_net(decom_r_low, decom_i_low, training=False)
    
    h, w, _ = tf.shape(input_decom)[1], tf.shape(input_decom)[2], tf.shape(input_decom)[3]
    i_low_data_ratio = tf.ones([h, w]) * input_low_i_ratio
    i_low_ratio_expand = tf.expand_dims(i_low_data_ratio, axis=2)
    i_low_ratio_expand2 = tf.expand_dims(i_low_ratio_expand, axis=0)
    
    adjust_i, A = Illumination_adjust_curve_net_ratio(decom_i_low, i_low_ratio_expand2)
    
    decom_r_sq = tf.squeeze(decom_r_low)
    r_gray = tf.image.rgb_to_grayscale(decom_r_sq)
    r_gray_gaussian = tf.image.gaussian_filter2d(r_gray, filter_shape=(3, 3), sigma=3)
    low_i = tf.minimum(tf.sqrt(r_gray_gaussian * 2), 1)
    low_i_expand_3 = tf.expand_dims(low_i, axis=3)
    result_denoise = restoration_r * low_i_expand_3
    result_denoise = restoration_r * low_i_expand_3
    fusion4 = result_denoise * adjust_i
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        # savemode
        checkpoint_dir = './experiment/exp2/checkpoint'
        illmin_name = "illumination_adjust_curve_net_global_rm_del_rotate"

        # Create a session and make it the default session
        sess = tf.Session()
        tf.keras.backend.set_session(sess)
        var_Decom = [var for var in tf.trainable_variables() if 'DecomNet' in var.name]
        var_adjust = [var for var in tf.trainable_variables() if 'I_enhance_Net' in var.name]
        var_restoration = [var for var in tf.trainable_variables() if 'Denoise_Net' in var.name]
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        var_restoration += bn_moving_vars
        saver_Decom = tf.train.Saver(var_list=var_Decom)
        saver_adjust = tf.train.Saver(var_list=var_adjust)
        saver_restoration = tf.train.Saver(var_list=var_restoration)
        decom_checkpoint_dir = os.path.join(checkpoint_dir, 'decom_net_retrain')
        ckpt_pre = tf.train.get_checkpoint_state(decom_checkpoint_dir)
        if ckpt_pre:
            print('[*] loaded ' + ckpt_pre.model_checkpoint_path)
            saver_Decom.restore(sess, ckpt_pre.model_checkpoint_path)
        else:
            print('[*] No decomnet pretrained model!')

        checkpoint_dir_adjust = os.path.join(checkpoint_dir, illmin_name)
        ckpt_adjust = tf.train.get_checkpoint_state(checkpoint_dir_adjust)
        if ckpt_adjust:
            print('[*] loaded ' + ckpt_adjust.model_checkpoint_path)
            saver_adjust.restore(sess, ckpt_adjust.model_checkpoint_path)
        else:
            print("[*] No adjust net pretrained model!")

        checkpoint_dir_restoration = os.path.join(checkpoint_dir, 'new_restoration_retrain')
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir_restoration)
        if ckpt:
            print('[*] loaded ' + ckpt.model_checkpoint_path)
            saver_restoration.restore(sess, ckpt.model_checkpoint_path)
        else:
            print("[*] No restoration net pretrained model!")

        # feed_dict = {
        #     input_decom: input_low_eval,
        #     input_low_i_ratio: ratio,
        #     # ... 添加其他的 feed_dict 值
        # }
        # output = sess.run(fusion4, feed_dict=feed_dict)
        # saver.save(sess, "model/model.ckpt")
